In [ ]:
#default_exp pde

In [ ]:
#exporti
import torch
import warnings
import numpy as np
from scipy.sparse import diags, csc_matrix

from dl4to.pde import SparseLinearSolver, PDESolver, FDMDerivatives, FDMAdjointDerivatives, FDMAssembly
from dl4to.utils import get_σ_vm

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Unpadded FDM solver

In [ ]:
#export
class UnpaddedFDM(PDESolver):
    """
    A PDE solver for linear elasticity that uses the finite differences method (FDM) with padding.
    """
    def __init__(self, θ_min:float=1e-6, # The minimal value in the stiffness matrix. For numerical reasons we can not allow 0s, since they may lead to singular matrices.
                 use_forward_differences:bool=True, # Whether to use forward differences or central differences.
                 assemble_tensors_when_passed_to_problem:bool=True, # Whether the PDE solver methods pre-assembles any tensors or arrays before solving the PDE for a concrete problem.
                 ):
        self._θ_min = θ_min
        self._linear_solver = SparseLinearSolver(use_umfpack=True, factorize=True)
        self.use_forward_differences = use_forward_differences
        self.assemble_tensors_when_passed_to_problem = assemble_tensors_when_passed_to_problem
        self.assembled_tensors = False
        super().__init__(assemble_tensors_when_passed_to_problem)


    @property
    def problem(self):
        return self._problem


    @property
    def shape(self):
        return self.problem.shape


    @property
    def Ω_dirichlet(self):
        return self.problem.Ω_dirichlet


    @property
    def θ_min(self):
        return self._θ_min


    @property
    def b(self):
        return self._b


    @property
    def h(self):
        return self.problem.h


    @property
    def linear_solver(self):
        return self._linear_solver


    def assemble_tensors(self, 
                         problem:"dl4to.problem.Problem" # The problem for which the tensors should be assembled.
                        ):
        """
        Assembles all FDM tensors from the problem object that can be pre-built without knowledge of the density distribution `θ`. This may take some time but makes future PDE evaluations for this problem much faster.
        """
        self._problem = problem.clone()
        GJ = lambda u: self._G(self._J(u))
        self._Ω_dirichlet_diags = diags(self.Ω_dirichlet.flatten().int().numpy())
        self._Jt_mat = FDMAssembly.assemble_operator(
            operator=self._J, shape=self.shape, 
            Ω_dirichlet=self.Ω_dirichlet, 
            filter_shape=3).transpose()
        self._GJ_mat = FDMAssembly.assemble_operator(
            operator=GJ, shape=self.shape, Ω_dirichlet=self.Ω_dirichlet, filter_shape=3)
        self._b = self._get_b()
        self.assembled_tensors = True


    def _get_θ_from_solution(self, solution, binary=False, clone=False):
        if clone:
            θ = solution.get_θ(binary).clone()
        else:
            θ = solution.get_θ(binary)
        return θ


    def _J(self, u, dirichlet=False):
        J = lambda u: torch.cat([
            FDMDerivatives.du_dx(u, self.h, self.use_forward_differences),
            FDMDerivatives.du_dy(u, self.h, self.use_forward_differences),
            FDMDerivatives.du_dz(u, self.h, self.use_forward_differences)
        ], dim=0)

        if dirichlet:
            return FDMAssembly.apply_dirichlet_zero_columns_to_operator(J, self.Ω_dirichlet)(u)
        return J(u)


    def _J_adj(self, σ, dirichlet=False):
        Jt = lambda σ: FDMAdjointDerivatives.du_dx_adj(σ[:3],  self.h, self.use_forward_differences) + \
                       FDMAdjointDerivatives.du_dy_adj(σ[3:6], self.h, self.use_forward_differences) + \
                       FDMAdjointDerivatives.du_dz_adj(σ[6:],  self.h, self.use_forward_differences)

        if dirichlet:
            return FDMAssembly.apply_dirichlet_zero_rows_to_operator(Jt, self.Ω_dirichlet)(σ)
        return Jt(σ)


    def _get_G(self):
        ν = self.problem.ν

        G = torch.tensor([
            [1-ν,  0-0,  0-0,    0-0,  ν-0,  0-0,    0-0,  0-0,  ν-0],
            [0-0, .5-ν,  0-0,   .5-ν,  0-0,  0-0,    0-0,  0-0,  0-0],
            [0-0,  0-0, .5-ν,    0-0,  0-0,  0-0,   .5-ν,  0-0,  0-0],

            [0-0, .5-ν,  0-0,   .5-ν,  0-0,  0-0,    0-0,  0-0,  0-0],
            [ν-0,  0-0,  0-0,    0-0,  1-ν,  0-0,    0-0,  0-0,  ν-0],
            [0-0,  0-0,  0-0,    0-0,  0-0, .5-ν,    0-0, .5-ν,  0-0],

            [0-0,  0-0, .5-ν,    0-0,  0-0,  0-0,   .5-ν,  0-0,  0-0],
            [0-0,  0-0,  0-0,    0-0,  0-0, .5-ν,    0-0, .5-ν,  0-0],
            [ν-0,  0-0,  0-0,    0-0,  ν-0,  0-0,    0-0,  0-0,  1-ν]
        ], dtype=self.problem.dtype)

        G = G.to(self.problem.device)
        return G / ((1 + ν) * (1 - 2 * ν))


    def _G(self, ε):
        ε = ε.type(self.problem.dtype)
        return torch.einsum('ij, jlmn -> ilmn', self._get_G(), ε)


    def _G_adj(self, σ):
        σ = σ.type(self.problem.dtype)
        return torch.einsum('ij, jlmn -> ilmn', self._get_G().t(), σ)


    def _GJ(self, u, dirichlet=False):
        apply_GJ = lambda u: self._G(self._J(u))

        if dirichlet:
            return FDMAssembly.apply_dirichlet_zero_columns_to_operator(apply_GJ, self.Ω_dirichlet)(u)
        return apply_GJ(u)


    def _GJ_adj(self, σ, dirichlet=False):
        apply_GJ_adj = lambda σ: self._J_adj(self._G_adj(σ))

        if dirichlet:
            return FDMAssembly.apply_dirichlet_zero_rows_to_operator(apply_GJ_adj, self.Ω_dirichlet)(σ)
        return apply_GJ_adj(σ)


    def _apply_θp(self, σ, θ, p=1., normalize=True):
        E = 1. if normalize else self.problem.E
        E_min = E * self.θ_min
        θ_ = E_min + θ**p * (E - E_min)
        return θ_ * σ


    def _assemble_θ(self, θ, p=1.):
        E = 1.
        E_min = E * self.θ_min
        θ_ = E_min + (θ**p).flatten().repeat(9).detach().numpy() * (E - E_min)
        return diags(θ_)


    def _A(self, u, θ, dirichlet=True, p=1.):
        u = u.view(3, θ.shape[-3], θ.shape[-2], θ.shape[-1])
        y = self._GJ(u, dirichlet)
        y = self._apply_θp(y, θ, p)
        y = self._J_adj(y, dirichlet)

        if dirichlet:
            y[self.Ω_dirichlet] = u.clone()[self.Ω_dirichlet]
        return y


    def _A_adj(self, y, θ, dirichlet=True, p=1.):
        y = y.view(3, θ.shape[-3], θ.shape[-2], θ.shape[-1])
        u = self._J(y, dirichlet)
        u = self._apply_θp(u, θ, p)
        u = self._GJ_adj(u, dirichlet)

        if dirichlet:
            u[self.Ω_dirichlet] = y.clone()[self.Ω_dirichlet]
        return u


    def _assemble_A(self, θ, p=1.):
        θ_mat = self._assemble_θ(θ, p)
        A = self._Jt_mat.dot(θ_mat.dot(self._GJ_mat)) + self._Ω_dirichlet_diags
        return csc_matrix(A)


    def _get_b(self):
        b = self.problem.F
        b[self.Ω_dirichlet] = 0
        b /= self.problem.E
        return b


    def _get_u(self, solution, p=1., binary=False):
        if binary and (solution.u_binary is not None):
            return solution.u_binary

        if (not binary) and (solution.u is not None):
            return solution.u

        if not self.assembled_tensors:
            self.assemble_tensors(solution.problem)

        θ = self._get_θ_from_solution(solution, binary=binary, clone=True)
        θ = θ.clamp(self.θ_min, 1)
        A_op = lambda u, θ: self._A(u, θ, p=p)
        A_mat = self._assemble_A(θ.cpu(), p)
        u = self._linear_solver(θ.cpu(), A_op, self.b.flatten(), A_mat)
        u = u.view(3, θ.shape[-3], θ.shape[-2], θ.shape[-1]).to(θ.device)

        if binary:
            solution.u_binary = u.clone()
        else:
            solution.u = u.clone()

        return u


    def _get_σ(self, solution, p=1., u=None, binary=False):
        if u is None:
            u = self._get_u(solution, p=p, binary=binary)

        θ = self._get_θ_from_solution(solution, binary=binary, clone=False)
        ε = self._J(u)
        σ = self._G(ε)
        σ = self._apply_θp(σ, θ, p=1., normalize=False)
        return σ


    def _stack_if_tensor_else_return_none(self, list):
        if any(type(entry) is not torch.Tensor for entry in list):
            return None
        return torch.stack(list)


    def solve_pde(self, 
                 solution:"dl4to.solution.Solution", # The solution for which the PDE should be solved.
                 p:float=1., # The SIMP exponent when solving the PDE. Should usually be left at its default value of `1.`.
                 binary:bool=False # Whether the densities in the solution should be binarized before solving the PDE.
                ):
        """
        Solves the pde for `solution` and SIMP exponent `p`. Returns three `torch.Tensor` objects: displacements `u`, stresses `σ` and von Mises stresses `σ_vm`.
        """
        u = self._get_u(solution, p=p, binary=binary)
        σ = self._get_σ(solution, p=p, u=u, binary=binary)
        σ_vm = get_σ_vm(σ)
        return u, σ, σ_vm

In [ ]:
#hide
from dl4to.solution import Solution
from dl4to.datasets import BasicDataset

In [ ]:
#hide
atol = 1e-3
dtype = torch.float64

In [ ]:
#hide
def get_rand_θ(problem):
    θ = torch.rand(1,*problem.shape).clamp_(problem.pde_solver.θ_min, 1)
    θ = θ**5

    θ[problem.Ω_design == 0] = 0
    θ[problem.Ω_design == 1] = 1
    return θ


def get_mock_objects(force=-1.5e5, resolution=35):
    """Mock Objects with Padding Depth 0"""
    problem = BasicDataset(resolution=resolution, dtype=dtype).ledge(force_per_area=force)
    problem.pde_solver = UnpaddedFDM()
    θ = get_rand_θ(problem)
    solution = Solution(problem, θ, enforce_θ_on_Ω_design=False)

    shape_prod = np.prod(problem.shape)
    u = torch.randn(3, *problem.shape, dtype=dtype)

    return problem, problem.pde_solver, θ, solution, shape_prod, u

In [ ]:
%%time
#hide

def test_create_FDM_object_and_get_properties():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    assert np.all(fdm.shape == problem.shape), fdm.shape
    assert torch.all(fdm.Ω_dirichlet == problem.Ω_dirichlet)

test_create_FDM_object_and_get_properties()

CPU times: user 500 ms, sys: 30.9 ms, total: 531 ms
Wall time: 554 ms


In [ ]:
%%time
#hide

def test_that_A_op_and_A_mat_sum_coincide():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()
    u *= 1e-6

    Au_op = fdm._A(u, θ, dirichlet=True)
    Au_mat = torch.tensor(fdm._assemble_A(θ).dot(u.flatten()))
    torch.allclose(Au_op.flatten().abs().sum(), Au_mat.flatten().abs().sum(), rtol=0), Au_op.flatten().abs().sum()


test_that_A_op_and_A_mat_sum_coincide()

CPU times: user 549 ms, sys: 22.4 ms, total: 572 ms
Wall time: 580 ms


In [ ]:
%%time
#hide

def test_that_u_solves_linear_system():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = fdm(solution)[0]
    Au = fdm._A(u, θ)
    assert Au.shape == fdm.b.shape
    assert torch.allclose(Au, fdm.b, atol=1e-2, rtol=0), torch.norm(Au - fdm.b)

test_that_u_solves_linear_system()

CPU times: user 11.4 s, sys: 17.9 ms, total: 11.4 s
Wall time: 1.85 s


In [ ]:
%%time
#hide

def test_that_u_solves_linear_system_in_norm():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = fdm(solution)[0]
    norm_error = (fdm._A(u, θ) - fdm.b).norm()
    norm_b = fdm.b.norm()
    assert norm_error / norm_b < 1e-1, norm_error / norm_b

test_that_u_solves_linear_system_in_norm()

CPU times: user 11.9 s, sys: 52.2 ms, total: 12 s
Wall time: 1.98 s


In [ ]:
%%time
#hide

def test_that_u_grows_linearly_with_applied_forces():
    force=-1.5e5
    problem, fdm, θ, solution, shape_prod, _ = get_mock_objects(force=force)
    u = fdm(solution)[0]

    n = torch.randint(0, 100, (1,))
    problem2, fdm2, θ2, solution2, shape_prod2, _ = get_mock_objects(force=force * n)

    solution2 = Solution(problem2, θ)
    u2 = fdm2(solution2)[0]

    assert torch.allclose(n * u, u2)


test_that_u_grows_linearly_with_applied_forces()

CPU times: user 22.8 s, sys: 31.4 ms, total: 22.9 s
Wall time: 3.61 s


In [ ]:
%%time
#hide

def test_if_shapes_are_correct():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    assert fdm._Jt_mat.shape == (3*shape_prod, 9*shape_prod)
    assert fdm._GJ_mat.shape == (9*shape_prod, 3*shape_prod)
    assert fdm._Ω_dirichlet_diags.shape == (3*shape_prod, 3*shape_prod)
    assert fdm._get_G().shape == (9,9)
    assert fdm._assemble_θ(θ).shape == (9*shape_prod, 9*shape_prod)
    assert fdm._assemble_A(θ).shape == (3*shape_prod, 3*shape_prod)


test_if_shapes_are_correct()

CPU times: user 534 ms, sys: 24.1 ms, total: 558 ms
Wall time: 572 ms


In [ ]:
%%time
#hide

def test_if_G_and_Gt_are_adjoint():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = torch.rand(9, *problem.shape, dtype=dtype)
    v = torch.rand(9, *problem.shape, dtype=dtype)
    Gu = fdm._G(u)
    Gtv = fdm._G_adj(v)
    assert torch.allclose(torch.dot(u.flatten(), Gtv.flatten()), torch.dot(Gu.flatten(), v.flatten()), atol=1e-2, rtol=0)

test_if_G_and_Gt_are_adjoint()

CPU times: user 608 ms, sys: 15.9 ms, total: 624 ms
Wall time: 582 ms


In [ ]:
%%time
#hide

def test_if_J_and_Jt_are_adjoint():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects(resolution=50)

    u = torch.rand(3, *problem.shape, dtype=dtype)
    v = torch.rand(9, *problem.shape, dtype=dtype)
    Ju = fdm._J(u, dirichlet=True)
    Jtv = fdm._J_adj(v, dirichlet=True)
    assert torch.allclose(torch.dot(u.flatten(), Jtv.flatten()), torch.dot(Ju.flatten(), v.flatten()), atol=1e-6, rtol=0)

test_if_J_and_Jt_are_adjoint()

CPU times: user 16.2 s, sys: 65.7 ms, total: 16.3 s
Wall time: 2.83 s


In [ ]:
%%time
#hide

def test_if_GJ_and_GJt_are_adjoint():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = torch.rand(3, *problem.shape, dtype=dtype)
    v = torch.rand(9, *problem.shape, dtype=dtype)
    GJu = fdm._GJ(u, dirichlet=True)
    GJtv = fdm._GJ_adj(v, dirichlet=True)
    assert torch.allclose(torch.dot(u.flatten(), GJtv.flatten()), torch.dot(GJu.flatten(), v.flatten()), atol=1e-2, rtol=0)

test_if_GJ_and_GJt_are_adjoint()

CPU times: user 520 ms, sys: 16 ms, total: 536 ms
Wall time: 548 ms


In [ ]:
%%time
#hide

def test_if_A_and_At_are_adjoint():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = torch.rand(3, *problem.shape, dtype=dtype) / 1e5
    v = torch.rand(3, *problem.shape, dtype=dtype) / 1e5
    Au = fdm._A(u, θ , dirichlet=True)
    Atv = fdm._A_adj(v, θ, dirichlet=True)
    assert torch.allclose(torch.dot(u.flatten(), Atv.flatten()), torch.dot(Au.flatten(), v.flatten()), atol=1e-1, rtol=0)

test_if_A_and_At_are_adjoint()

CPU times: user 523 ms, sys: 12 ms, total: 535 ms
Wall time: 544 ms


In [ ]:
%%time
#hide

def test_if_transposition_of_J_acts_as_adjoint_in_matrix_case():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = torch.rand(3*shape_prod, 1, dtype=dtype)
    v = torch.rand(9*shape_prod, 1, dtype=dtype)
    Jt = fdm._Jt_mat
    J = fdm._Jt_mat.transpose()
    Jtv = torch.tensor(Jt.dot(v))
    Ju = torch.tensor(J.dot(u))
    assert torch.allclose(torch.dot(u.flatten(), Jtv.flatten()), torch.dot(Ju.flatten(), v.flatten()), atol=1e-2)

test_if_transposition_of_J_acts_as_adjoint_in_matrix_case()

CPU times: user 514 ms, sys: 23.8 ms, total: 537 ms
Wall time: 548 ms


In [ ]:
%%time
#hide

def test_if_transposition_of_J_acts_as_adjoint_when_using_forward_differences_in_matrix_case():
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    u = torch.rand(3*shape_prod, 1, dtype=dtype)
    v = torch.rand(9*shape_prod, 1, dtype=dtype)
    Jt = fdm._Jt_mat
    J = fdm._Jt_mat.transpose()
    Jtv = torch.tensor(Jt.dot(v))
    Ju = torch.tensor(J.dot(u))
    assert torch.allclose(torch.dot(u.flatten(), Jtv.flatten()), torch.dot(Ju.flatten(), v.flatten()), atol=1e-2)


test_if_transposition_of_J_acts_as_adjoint_when_using_forward_differences_in_matrix_case()

CPU times: user 517 ms, sys: 8.01 ms, total: 525 ms
Wall time: 532 ms


In [ ]:
%%time
#hide

def test_if_assembly_of_GJ_is_correct(number=2):
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    GJ = lambda u: fdm._G(fdm._J(u))
    GJ_op = FDMAssembly.apply_dirichlet_zero_columns_to_operator(GJ, fdm.Ω_dirichlet)
    for _ in range(number):
        x = torch.randn(3,*fdm.shape, dtype=dtype)
        assert torch.allclose(torch.tensor(fdm._GJ_mat.todense(), dtype=dtype).mv(x.flatten()), GJ_op(x).flatten(), atol=1e-4)

test_if_assembly_of_GJ_is_correct()

CPU times: user 2.08 s, sys: 525 ms, total: 2.61 s
Wall time: 987 ms


In [ ]:
%%time
#hide

def test_if_assembly_of_Jt_is_correct(number=2):
    problem, fdm, θ, solution, shape_prod, u = get_mock_objects()

    J_op = FDMAssembly.apply_dirichlet_zero_columns_to_operator(fdm._J, fdm.Ω_dirichlet)

    for _ in range(number):
        x = torch.randn(3, *fdm.shape, dtype=dtype)
        assert torch.allclose(torch.tensor(fdm._Jt_mat.transpose().todense(), dtype=dtype).mv(x.flatten()), J_op(x).flatten(), atol=1e-4)

test_if_assembly_of_Jt_is_correct()

CPU times: user 2.06 s, sys: 464 ms, total: 2.52 s
Wall time: 901 ms


In [ ]:
%%time
#hide

def test_tensile_rod_solution_is_close_to_theoretical_case(tol=0.001):
    force_per_area = -1.5e5
    tensile_rod_problem = BasicDataset(resolution=30).tensile_rod(force_per_area=force_per_area)
    tensile_rod_problem.pde_solver = UnpaddedFDM()
    solution = tensile_rod_problem.trivial_solution
    u_pde, σ, _ = solution.solve_pde()
    relative_error = force_per_area / -σ[-1, 1, 1, 10].item()
    assert 1. - tol < relative_error < 1. + tol, relative_error

test_tensile_rod_solution_is_close_to_theoretical_case()

CPU times: user 263 ms, sys: 0 ns, total: 263 ms
Wall time: 265 ms


In [ ]:
%%time
#hide

def test_that_ledge_solution_is_close_to_theoretical_beam_displacement(tol=0.25):
    force_per_area = -1.5e5
    ledge_problem = BasicDataset(resolution=30).ledge(force_per_area=force_per_area)
    ledge_problem.pde_solver = UnpaddedFDM(use_forward_differences=True)
    solution = ledge_problem.trivial_solution
    u_pde, _, _ = solution.solve_pde()
    pde_max_displacement = u_pde.min().item()

    #theoretical solution
    l, b, h = ledge_problem.size[:]
    I = (b*h**3)/12
    force_per_length = force_per_area * b
    E = ledge_problem.E
    theoretical_max_displacement = force_per_length*(l**4) /(8*E*I)

    relative_error = abs(theoretical_max_displacement / pde_max_displacement)
    assert 1. - tol < relative_error < 1. + tol, relative_error

test_that_ledge_solution_is_close_to_theoretical_beam_displacement()

CPU times: user 4.75 s, sys: 17.8 ms, total: 4.77 s
Wall time: 861 ms


In [ ]:
%%time
#slow
#hide

def test_displacements_dont_propagate_through_void__check_forks_displacements_in_the_top_are_at_least_5times_greater_than_in_the_bottom():
    fork_problem = BasicDataset(resolution=35).fork()
    fork_problem.pde_solver = UnpaddedFDM()
    solution = fork_problem.trivial_solution
    solution.θ *= 1e-6
    u = solution.solve_pde()[0]

    displacements_in_the_top = u[2, -1, :, -1].mean().abs()
    displacements_in_the_bottom = u[2, -1, :, 1].mean().abs()

    assert displacements_in_the_top > 5 * displacements_in_the_bottom, displacements_in_the_top / displacements_in_the_bottom

test_displacements_dont_propagate_through_void__check_forks_displacements_in_the_top_are_at_least_5times_greater_than_in_the_bottom()

CPU times: user 10.1 s, sys: 47.9 ms, total: 10.1 s
Wall time: 1.48 s
